In [1]:
%pip install datasets
%pip install bashlex

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


# Shell Scripts


In [2]:
from datasets import load_dataset
from bashlex import parser, ast
from tqdm import tqdm

In [3]:
ds = load_dataset(
    "codeparrot/github-code", streaming=True, split="train", languages=["Shell"]
)

Using custom data configuration Shell-all-9563d1b2f034c487


# Shell Commands


In [4]:
dataset = []

https://gitlab.ics.muni.cz/muni-kypo-trainings/datasets/commands


In [5]:
import os
import json

In [6]:
data_dir_1 = "./commands"

In [7]:
file_paths = []
for root, dirs, files in os.walk(data_dir_1):
    for name in files:
        if name.endswith((".json")):
            full_path = os.path.join(root, name)
            file_paths.append(full_path)

In [8]:
for path in file_paths:

    with open(path, "r") as f:
        data = f.read()
        for line in data.split("\n"):
            if len(line) == 0:
                continue
            json_data = json.loads(line)
            if "cmd" in json_data:
                dataset.append(json_data["cmd"])

In [9]:
len(dataset)

21089

https://huggingface.co/datasets/spignelon/bash_history


In [10]:
data_dir_2 = "./bash.txt"

In [11]:
with open(data_dir_2, "r") as f:
    data = f.read()
    lines = data.split("\n")
    dataset.extend(lines)

In [12]:
len(dataset)

202059

https://huggingface.co/datasets/gayathrimanoj/dataset_shell


In [13]:
data_dir_3 = "./train.jsonl"
data_dir_4 = "./test.jsonl"

In [14]:
with open(data_dir_3, "r") as f:
    data = f.read()
    lines = data.split("\n")
    for line in lines:
        if len(line) == 0:
            continue
        json_data = json.loads(line)
        text = json_data["text"]
        text_split = text.split("[/INST]")[1]
        res = text_split.rstrip(" </s>")
        if res == "Invalid command":
            continue
        if res == "Invalid command.":
            continue
        if res == "Invalid Command":
            continue
        if res == "Invalid Command.":
            continue
        dataset.append(res)

In [15]:
with open(data_dir_4, "r") as f:
    data = f.read()
    lines = data.split("\n")
    for line in lines:
        if len(line) == 0:
            continue
        json_data = json.loads(line)
        text = json_data["text"]
        text_split = text.split("[/INST]")[1]
        res = text_split.rstrip(" </s>")
        if res == "Invalid command":
            continue
        if res == "Invalid command.":
            continue
        if res == "Invalid Command":
            continue
        if res == "Invalid Command.":
            continue
        dataset.append(res)

In [16]:
print(len(dataset))

202601


https://huggingface.co/datasets/byroneverson/shell-cmd-instruct


In [17]:
data_dir_5 = "./train (1).jsonl"

In [18]:
with open(data_dir_5, "r") as f:
    data = f.read()
    for line in data.split("\n"):
        if len(line) == 0:
            continue
        json_data = json.loads(line)
        if "cmd" in json_data:
            dataset.append(json_data["cmd"])

In [19]:
print(len(dataset))

203101


In [20]:
import pickle

In [21]:
with open("dataset.pkl", "wb") as fp:
    pickle.dump(dataset, fp)

# Preprocessing


In [22]:
import torch
import pytorch_forecasting as pl

/usr/local/lib/python3.9/dist-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: libtorch_cuda_cu.so: cannot open shared object file: No such file or directory
  warn(f"Failed to load image Python extension: {e}")


In [23]:
for i, cmd in enumerate(dataset):
    dataset[i] += "\n"

In [24]:
concatinated = " ".join(dataset)
chars = sorted(list(set(concatinated)))
vocab_size = len(chars)
stoi = {ch: i for i, ch in enumerate(chars)}
itos = {i: ch for i, ch in enumerate(chars)}
encode = lambda s: [stoi[c] for c in s]
decode = lambda l: "".join([itos[i] for i in l])

with open("stoi.pkl", "wb") as fp:
    pickle.dump(stoi, fp)

with open("itos.pkl", "wb") as fp:
    pickle.dump(itos, fp)

In [25]:
encoded_dataset = []
for cmd in dataset:
    encoded_dataset.append(torch.tensor(encode(cmd)))

In [28]:
block_size = 356

In [29]:
x = pl.utils.padded_stack(
    [d[:-1][:block_size] for d in encoded_dataset], "left", "constant", 3
)
y = pl.utils.padded_stack(
    [d[1:][:block_size] for d in encoded_dataset], "left", "constant", 3
)

In [30]:
with open("dataset_x.pt", "wb") as fp:
    torch.save(x, fp)

with open("dataset_y.pt", "wb") as fp:
    torch.save(y, fp)